In [0]:
%sh
#ls -al 
cd /Volumes/workspace/aero/flights
unzip flights.zip
ls -al

In [0]:
file_location = f'/Volumes/workspace/aero/flights/flights.csv'
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)
